# Exercise 01

## Task 1. A simple SIR model

In the first lecture you were introduced to a simple epidemiological model, the so-called SIR model, defined by

$$
\frac{dS}{dt} = - \beta \frac{S I}{N}, \quad \frac{dI}{dt} = \beta \frac{S I}{N} - \gamma I, \quad \frac{dR}{dt} = \gamma I;
$$

### Task 1.1

Assume a total population of $ N = 1000 $. Define a SIR model with $ \beta = 2.4 $, $ \gamma = 1/20 $, one initial sick patient, and 60 percent of the population susceptible to illness, and plot its time evolution until the dynamics run into a fixed point.

### Task 1.2

Adjust the model for the case that infections do not confer long-term immunity to the disease (this means that recovered patients instead fall back to the susceptible group). Depending on different values for $ \beta $ and $ \gamma $, what are the two types of long-term behavior that this model predicts?

### Task 1.3

Assume a mortality rate of 1 percent, and include deceased patients into the model. Plot the time evolution of the new model.

## Task 2. Higher order ODE systems

Show that any $n$-th order ODE of the form $ f(t, x, \frac{dx}{dt}, \frac{d^2x}{dt^2}, \cdots, \frac{d^nx}{dt^n}) = 0 $ can always be transformed into a system of $n$ first-order ODEs.

## Task 3. ODEs on the real line

Explain/show why one-dimensional ODEs on the real line can never give rise to oscillatory behavior.

## Task 4. Flow field of linear ODEs

Assume a linear ODE system, defined by 

$$
\frac{dx}{dt} = A \cdot \mathbf{x}, \quad \text{where} \quad A = \begin{pmatrix} 0.1 & -0.3 \\ 0.2 & -0.3 \end{pmatrix}, \quad \text{and} \quad \mathbf{x} = \begin{pmatrix} x_1 \\ x_2 \end{pmatrix}
$$

### Task4.1

Determine the fixed points of this system by hand.

### Task4.2

Plot the time evolution of the system for an initial vector of $ \mathbf{x_0} = \begin{pmatrix} 1 \\ 0 \end{pmatrix} $.

### Task4.3

Plot the flow field of the system in an appropriate range. What do you observe?

### Task4.4

In the lecture you learned about several different types of dynamics (see also figure 5.2.8. in the Strogatz book) that can be obtained from a two-dimensional linear dynamical system (unstable node, stable node, saddle point, line attractor, degenerate node, stable spiral, unstable spiral and center). Define system parameters of your choosing that satisfy the respective requirements, and plot the corresponding trajectories and flow fields.